In [11]:
system_content = """Based on the attached contract, please generate the invoice schedule starting from the initial invoice which is due upon effective date and ending at the end of 2026. In the invoice schedule, for each invoice please capture the following items:

Customer Name
Invoice Number (formatted as first three letters of customer name, then dash, then number)
Invoice Date (in mm/dd/yyyy format)
Due Date (in mm/dd/yyyy format)
Product Name
Price (in plain number)

Please provide it in JSON format.
"""



In [13]:
generate_invoice_schedule = {
    "name": "generate_invoice_schedule",
    "description": "Creates an invoice schedule from a given contract text, detailing each invoice from the contract's effective date until the end of 2026. This includes customer details, invoice numbering, product descriptions, and pricing.",
    "parameters": {
        "type": "object",
        "properties": {
            "metadata": {
                # customer: orm.Mapped[Optional[str]] 
        # effective_date: orm.Mapped[Optional[datetime]] = orm.mapped_column(type_=TIMESTAMP(timezone=True))
        # contract_value: orm.Mapped[Optional[float]]
        # frequency: orm.Mapped[Optional[str]]
        # price_escalator: orm.Mapped[Optional[float]]  # Assuming percentage form
        # payment_terms: orm.Mapped[Optional[str]]
        # filepath = orm.Mapped[Optional[str]]
        # billing_contact: orm.Mapped[Optional[str]]
        # product:
            "invoice_items": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "customer_name": {
                            "type": "string",
                            "description": "The name of the customer as stated in the contract.",
                        },
                        "invoice_number": {
                            "type": "string",
                            "description": "A unique identifier for the invoice, typically starting with the first three letters of the customer's name followed by a dash and a sequence number.",
                        },
                        "invoice_date": {
                            "type": "string",
                            "format": "date",
                            "description": "The date on which the invoice is issued, formatted as mm/dd/yyyy.",
                        },
                        "due_date": {
                            "type": "string",
                            "format": "date",
                            "description": "The date by which the invoice must be paid, formatted as mm/dd/yyyy.",
                        },
                        "product_name": {
                            "type": "string",
                            "description": "The name of the product or service being billed in the invoice.",
                        },
                        "price": {
                            "type": "number",
                            "description": "The amount to be paid for the product or service, represented as a plain number without formatting.",
                        },
                    },
                    "required": ["customer_name", "invoice_number", "invoice_date", "due_date", "product_name", "price"],
                },
                "description": "Details each invoice within the schedule, including customer details, dates, product information, and pricing.",
            },
        },
        "required": ["contract_text", "invoice_items"],
    },
}


In [15]:
    
from langchain_community.callbacks import get_openai_callback
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.prompts  import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

from app import create_app, db
from app.models import Document
app = create_app()
with app.app_context():
    document = db.session.get(Document, 2)
    document_text = document.full_text
    
functions = [generate_invoice_schedule]
model = "gpt-4"
llm = ChatOpenAI(model = model)
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=(system_content)),
        HumanMessagePromptTemplate.from_template("{document_text}"),
    ])
chain =  prompt | llm.bind(function_call={"name": generate_invoice_schedule["name"]}, functions=functions) | JsonOutputFunctionsParser()
schedule = chain.invoke({"document_text":document_text})

In [18]:
import datetime
from datetime import datetime, timezone

datetime_fields = ['invoice_date', 'due_date']
for item in schedule["invoice_items"]:
    for field in datetime_fields:
        if field in item and item[field] and item[field] != 'null':
            # Parse the 'MM/DD/YYYY' datetime string to a datetime object
            dt = datetime.strptime(item[field], '%m/%d/%Y')
            # Assume the datetime is in UTC and make it timezone-aware
            item[field] = dt.replace(tzinfo=timezone.utc)
    
schedule

{'contract_text': 'Statement of Work — Column Platform\n\nThis Statement of Work (“SOW”), adopts and incorporates by reference the terms and\nconditions of the Master Services Agreement (the “Agreement”), entered into by both Parties,\nbetween Column, a Delaware Limited Liability Company (“Column”) and SynthCorp Systems, a\nCalifornia LLC (“Client,”), as it may be amended from time to time. Column and Client may each be\nreferred to as a “Party” or, collectively, as the “Parties.”\n\nThis SOW is effective beginning on the date set forth on the signature page hereof (“SOW\nEffective Date”) and will remain in effect until it expires or is earlier terminated in accordance with\nthis SOW. The software provided under this SOW will be rendered in accordance with and be\nsubject to the terms and conditions of this SOW, the Agreement, and any related schedules and\nexhibits. The Parties agree that if there is any conflict between the terms and conditions of the\nAgreement and the terms and con

In [24]:
import stripe
import time

# Set your secret API key
stripe.api_key = 'sk_test_51OyHP6HtST3RqXtdJtRIwKee03URhNf3GRr1DMkPLBdgYDZIgHswQgdVGe31crOjIP3JRbepwi6dSzGsJ8tJHIUZ00kVKyMNmR'

customer = "cus_Q0GAyIdHPixUMv"
# Create the invoice item
invoice = stripe.Invoice.create(
    customer=customer,  # Replace with your customer's ID
    auto_advance=True,             # Automatically finalize and pay the invoice
    collection_method='send_invoice',
    days_until_due=30              # Set the number of days until the invoice is due
)
time.sleep(2)
stripe.InvoiceItem.create(
    customer=customer,  # Replace with your customer's ID
    amount=775000,  
    invoice=invoice.id,
    currency='usd',
    description='One-time setup fee',
)

# Create and send the invoice

# Finalize and send the invoice
stripe.Invoice.finalize_invoice(invoice.id)
stripe.Invoice.send_invoice(invoice.id)

print(f"Invoice {invoice.id} created and sent.")

Invoice in_1PAMOvHtST3RqXtdxFQS8PZr created and sent.


In [25]:
invoice

<Invoice invoice id=in_1PAMOvHtST3RqXtdxFQS8PZr at 0x126223480> JSON: {
  "account_country": "US",
  "account_name": "Exante",
  "account_tax_ids": null,
  "amount_due": 0,
  "amount_paid": 0,
  "amount_remaining": 0,
  "amount_shipping": 0,
  "application": null,
  "application_fee_amount": null,
  "attempt_count": 0,
  "attempted": false,
  "auto_advance": true,
  "automatic_tax": {
    "enabled": false,
    "liability": null,
    "status": null
  },
  "billing_reason": "manual",
  "charge": null,
  "collection_method": "send_invoice",
  "created": 1714267173,
  "currency": "usd",
  "custom_fields": null,
  "customer": "cus_Q0GAyIdHPixUMv",
  "customer_address": null,
  "customer_email": "giedrius+edgedb@exante.app",
  "customer_name": "EdgeDB",
  "customer_phone": null,
  "customer_shipping": null,
  "customer_tax_exempt": "none",
  "customer_tax_ids": [],
  "default_payment_method": null,
  "default_source": null,
  "default_tax_rates": [],
  "description": null,
  "discount": null